In [14]:
import pandas as pd
import numpy as np
import optuna 
import sklearn

from sklearn.metrics import ( root_mean_squared_error, 
                             mean_absolute_error, 
                             mean_absolute_percentage_error )

In [15]:
DIR = "data/postproccessed/"
# leemos x de fichero
X_train = pd.read_pickle(DIR + "X_train_pipe_1.pkl")
X_test = pd.read_pickle(DIR + "X_test_pipe_1.pkl")
# leemos y de fichero
y_train = pd.read_pickle(DIR + "y_train_pipe_1.pkl")
y_test = pd.read_pickle(DIR + "y_test_pipe_1.pkl")

In [22]:
y_train.values.flatten().shape

(20974,)

In [34]:
def objective(trial):
    x, y = X_train.values, y_train.values.flatten()
    n_estimators = trial.suggest_int("n_estimators", 10, 500, log=True)
    max_depth = trial.suggest_int("max_depth",2,32, log=True)
    classifier_name = trial.suggest_categorical("classifier", ["GradientBoost", "RandomForest", "HistGradientBoost"])
    if classifier_name == "GradientBoost":
        learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1, log=True)
        
        estimator = sklearn.ensemble.GradientBoostingRegressor(
            n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate)
    if classifier_name == "HistGradientBoost":
        learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1, log=True)
        
        n_bins = trial.suggest_int("n_bins", 10, 255, log=True)
        estimator = sklearn.ensemble.HistGradientBoostingRegressor(
            max_iters=n_estimators, max_depth=max_depth, learning_rate=learning_rate)
    else:
       
        min_samples_split = trial.suggest_int("min_samples_split",5,50, log=True)
        estimator = sklearn.ensemble.RandomForestRegressor(
            max_depth=max_depth, min_samples_split=min_samples_split, n_estimators=n_estimators
        )

    score = sklearn.model_selection.cross_val_score(estimator, x, y, scoring='neg_mean_absolute_percentage_error', n_jobs=-1, cv=3)
    result = score.mean()
    return result

In [36]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)
print(study.best_trial)

[I 2025-03-24 16:13:41,347] A new study created in memory with name: no-name-489ab466-04bf-4d82-84ca-8256d31b04aa
[W 2025-03-24 16:13:41,349] Trial 0 failed with parameters: {'n_estimators': 46, 'classifier': 'HistGradientBoost', 'learning_rate': 0.007922713085620564, 'n_bins': 117} because of the following error: UnboundLocalError("cannot access local variable 'max_depth' where it is not associated with a value").
Traceback (most recent call last):
  File "/Users/mmartin/opt/anaconda3/envs/modelizacion_datos/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/_0/0zwytxv51nb3c6wxtthk3bwh0000gn/T/ipykernel_67476/4068622762.py", line 14, in objective
    max_iters=n_estimators, max_depth=max_depth, learning_rate=learning_rate)
                                      ^^^^^^^^^
UnboundLocalError: cannot access local variable 'max_depth' where it is not associated with a value

UnboundLocalError: cannot access local variable 'max_depth' where it is not associated with a value